
# 🧪 C5.2.2 — Sélection de variables (Bloc 5)

**Objectif :** choisir et justifier des **méthodes de sélection de variables** pour réduire la dimension, améliorer la
généralisation et l'explicabilité. On compare **plusieurs familles** :  
- **Filtre (statistique)** : test **chi²** sur TF‑IDF (non‑négatif)  
- **Méthode incorporée (embedded)** : **L1** (LASSO) via `LogisticRegression` + `SelectFromModel`  
- **Réduction de dimension** : **TruncatedSVD** (LSA) sur matrices creuses  
*(option)* **RFE** sur un sous‑ensemble pour illustrer l'élimination récursive

On s'appuie sur les features générés en **C5.2.1**.



## 🎙️ Discours (1–2 min)
> « Nous retenons un triptyque complémentaire : **chi²** (rapide, interpretable), **L1** (sélection parcimonieuse embarquée),
> et **SVD** (compression thématique). Nous comparons l'impact sur AP/F1 et le **nombre de variables** conservées,
> puis nous **sérialisons** les sélecteurs pour l'entraînement (C5.2.3). »


## ⚙️ 0) Configuration & chemins

In [1]:

from pathlib import Path

# Artefacts de C5.2.1
X_PATH = Path("data/processed/X_tfidf_sample.npz")
Y_PATH = Path("data/processed/y_binary_sample.joblib")
FEAT_ART = Path("models")  # on prendra le plus récent features_tfidf_*.joblib

# Paramètres sélection
CHI2_K = 50_000            # k pour SelectKBest(chi2)
L1_C   = 0.5               # régularisation L1 (plus petit => plus de shrinkage)
SVD_K  = 300               # nb composantes SVD

DO_RFE = False             # RFE très coûteux en très haute dimension (False par défaut)
RFE_N_FEATURES = 5_000     # si DO_RFE=True, cible finale (après préfiltre)

RANDOM_STATE = 42

## 📥 1) Chargement X/y + artefacts de noms

In [2]:

import json, re, time
import numpy as np
import pandas as pd
from joblib import load
from scipy import sparse

assert X_PATH.exists() and Y_PATH.exists(), "Exécute d'abord C5.2.1 pour produire X/y."
X = sparse.load_npz(X_PATH)
y = load(Y_PATH)

# Récupérer le plus récent artefact features_tfidf_*.joblib
feat_jobs = sorted(FEAT_ART.glob("features_tfidf_*.joblib"))
assert feat_jobs, "Artefact features_tfidf_*.joblib introuvable (C5.2.1)."
feat_job = feat_jobs[-1]
art = load(feat_job)

# Construire la liste des noms de features (mot + char + meta)
names = []
if art.get("vectorizer_word") is not None:
    names += list(art["vectorizer_word"].get_feature_names_out())
if art.get("vectorizer_char") is not None:
    names += list(art["vectorizer_char"].get_feature_names_out())
if art.get("feature_meta_cols"):
    names += list(art["feature_meta_cols"])
feature_names = np.array(names, dtype=object)
print("X shape:", X.shape, "| y:", y.shape, "| nb_features:", len(feature_names), "| artefact:", feat_job.name)

ModuleNotFoundError: No module named 'numpy'

## ✂️ 2) Split train/test (stratifié, reproductible)

In [5]:

from sklearn.model_selection import train_test_split

Xtr, Xte, ytr, yte = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
Xtr.shape, Xte.shape, ytr.mean(), yte.mean()

((96000, 180007), (24000, 180007), 0.7614583333333333, 0.7614583333333333)

## 🧰 3) Utilitaire d'évaluation rapide

In [6]:

import time, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, f1_score, precision_recall_curve

def eval_pipeline(Xtr, Xte, ytr, yte, desc: str):
    t0 = time.time()
    clf = LogisticRegression(max_iter=2000, class_weight="balanced")
    clf.fit(Xtr, ytr)
    fit_s = time.time() - t0
    proba = clf.predict_proba(Xte)[:,1]
    ap = average_precision_score(yte, proba)
    # F1@0.5 et F1 optimal
    prec, rec, thr = precision_recall_curve(yte, proba)
    f1 = (2*prec*rec)/(prec+rec+1e-9)
    best_idx = int(np.nanargmax(f1))
    thr_star = float(thr[best_idx]) if best_idx < len(thr) else 0.5
    f1_star = float(f1[best_idx])
    f1_05 = f1_score(yte, proba>=0.5)
    print(f"[{desc}] AP={ap:.3f} | F1@0.5={f1_05:.3f} | F1@t*={f1_star:.3f} (t*={thr_star:.2f}) | fit={fit_s:.2f}s")
    return {"desc": desc, "AP": ap, "F1@0.5": f1_05, "F1@t*": f1_star, "t*": thr_star, "fit_s": fit_s}

## 🎚️ 4) Baseline sans sélection

In [7]:

import pandas as pd
res = []
res.append(eval_pipeline(Xtr, Xte, ytr, yte, desc=f"Baseline (no selection) — p={Xtr.shape[1]:,}"))
pd.DataFrame(res)

[Baseline (no selection) — p=180,007] AP=0.983 | F1@0.5=0.927 | F1@t*=0.938 (t*=0.31) | fit=342.86s


c:\Users\antoi\anaconda3\envs\monenv\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,desc,AP,F1@0.5,F1@t*,t*,fit_s
0,"Baseline (no selection) — p=180,007",0.982659,0.927284,0.937503,0.309827,342.864347


### 🔍 Sauvegarde des artefacts

In [12]:
# 💽 Sauvegarde du sélecteur chi² choisi + méta décision pour C5.2.3
from joblib import dump
from pathlib import Path
import time, json

# `sel` = votre SelectKBest(chi2, k=50_000) fit dans la cellule de comparaison
assert 'sel' in globals(), "Le sélecteur `sel` doit être créé plus haut."

Path("models").mkdir(parents=True, exist_ok=True)
stamp = time.strftime("%Y%m%d_%H%M%S")

# 1) sélecteur chi²
sel_path = Path("models")/f"selector_chi2_k{sel.k}_{stamp}.joblib"
dump({"chi2": sel, "k": int(sel.k)}, sel_path)

# 2) méta décision (pour tracer le choix dans le repo)
decision = {
    "chosen_pipeline": "tfidf -> chi2(k=50k) -> logreg(lbfgs, L2, C=1.0)",
    "k": int(sel.k),
    "logreg": {"solver": "lbfgs", "penalty": "l2", "C": 1.0, "max_iter": 5000, "tol": 1e-3},
    "notes": "Qualité ≈ baseline; fit ~2.5x plus rapide; calibration préservée (t*≈0.31).",
    "created": stamp
}
(Path("models")/f"selection_decision_{stamp}.json").write_text(json.dumps(decision, indent=2), encoding="utf-8")

print("Écrit →", sel_path)
print("Écrit →", f"models/selection_decision_{stamp}.json")


Écrit → models\selector_chi2_k50000_20250912_171611.joblib
Écrit → models/selection_decision_20250912_171611.json


# 📈 5) Filtre chi² — SelectKBest

In [17]:
from sklearn.feature_selection import SelectKBest, chi2

# k défini dans ta cellule de config (ex. CHI2_K=50_000)
k = min(CHI2_K, Xtr.shape[1])

# Sélecteur chi²
sel = SelectKBest(score_func=chi2, k=k).fit(Xtr, ytr)
Xtr_chi = sel.transform(Xtr)
Xte_chi = sel.transform(Xte)

print(f"chi² sélectionne k={k:,} features  |  Xtr_chi: {Xtr_chi.shape}  |  Xte_chi: {Xte_chi.shape}")

# (pour la cellule 5bis)
mask = sel.get_support()
selected_names = feature_names[mask]

chi² sélectionne k=50,000 features  |  Xtr_chi: (96000, 50000)  |  Xte_chi: (24000, 50000)


### 🔍 5bis) Top features (chi²) — termes les plus discriminants
> Interprétation : plus le **score χ²** est élevé, plus la variable discrimine les classes (0/1). On documente les 25 premiers.

### 🔍 Top features (chi²)

In [18]:
# --- Top features (chi²) à partir du sélecteur `sel` ---
import numpy as np, pandas as pd

scores_all = sel.scores_              # scores par feature (dans l'espace original)
mask = sel.get_support()              # booléen: features retenues
selected_names = feature_names[mask]  # noms des features retenues

# Nettoyage NaN/inf et tri décroissant
scores_sel = np.nan_to_num(scores_all[mask], nan=0.0, posinf=0.0, neginf=0.0)
order = np.argsort(scores_sel)[::-1]            # décroissant
top_n = 25
top_df = pd.DataFrame({
    "feature": selected_names[order][:top_n],
    "chi2_score": scores_sel[order][:top_n].astype(float)
})
top_df

,feature,chi2_score
0,review_len,623855.812100
1,word_count,115366.200711
2,helpful_vote,23897.490482
3,ques_count,1335.442057
4,bang_count,722.820268
5,great,580.975434
6,return,502.651629
7,stopped,487.038369
8,waste,434.491978
9,stopped working,421.230187


## 🧲 6) Méthode incorporée — L1 (LogReg)

In [20]:
## 🧲 6bis) Méthode incorporée — L1 sur l’espace chi²(k) + évaluation lbfgs
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
import numpy as np, pandas as pd

# 0) Sélection L1 (parcimonie) sur l’espace déjà filtré par chi²
l1_clf = LogisticRegression(
    penalty="l1", solver="saga", C=1.0,             # tu peux tester 0.5 / 1.0
    class_weight="balanced", max_iter=3000, tol=1e-3
)
sfm = SelectFromModel(l1_clf, threshold="median", prefit=False)   # ~50% des + fortes
Xtr_l1 = sfm.fit_transform(Xtr_chi, ytr)
Xte_l1 = sfm.transform(Xte_chi)
print(f"chi²(k={sel.k:,}) → L1(thr=median) → kept = {Xtr_l1.shape[1]:,} / {Xtr_chi.shape[1]:,}")

# 1) Évaluation du pipeline final sur ces variables (lbfgs reste meilleur chez toi)
res_l1 = eval_pipeline(
    Xtr_l1, Xte_l1, ytr, yte,
    desc=f"chi²(k={sel.k:,}) → L1(thr=median) → lbfgs"
)
pd.DataFrame([res_l1])

chi²(k=50,000) → L1(thr=median) → kept = 25,000 / 50,000
[chi²(k=50,000) → L1(thr=median) → lbfgs] AP=0.982 | F1@0.5=0.921 | F1@t*=0.936 (t*=0.27) | fit=157.48s


,desc,AP,F1@0.5,F1@t*,t*,fit_s
0,"chi²(k=50,000) → L1(thr=median) → lbfgs",0.981772,0.921264,0.935695,0.267476,157.477329


In [21]:
res_l1

{'desc': 'chi²(k=50,000) → L1(thr=median) → lbfgs',
 'AP': 0.9817723811867631,
 'F1@0.5': 0.921263877028181,
 'F1@t*': 0.935694514857695,
 't*': 0.26747602855195807,
 'fit_s': 157.47732949256897}

Lecture express (à coller sous la cellule – Markdown prêt)

chi²(50k) → L1(thr=median) → lbfgs

La méthode incorporée (L1) a sélectionné automatiquement ~25k variables parmi les 50k du pré-filtre chi².

On obtient un fit 2,4× plus rapide qu’avec chi²+lbfgs pour une perte très faible : ΔAP ≈ –0,002, ΔF1@0.5 ≈ –0,007.

Le seuil optimal passe de ~0,31 à ~0,27, signe d’une calibration encore correcte.

👉 Cette étape coche la case “méthode incorporée” (sélection pendant l’apprentissage) et fournit un pipeline light (25k features) utile si l’on vise moins de RAM/latence.

Décision pratique :

Qualité maximale & simplicité → garder TF-IDF → chi²(50k) → LogReg L2 (lbfgs).

Contraintes de temps/mémoire → envisager TF-IDF → chi²(50k) → L1 → LogReg L2, qui divise par ~2 le nombre de features sans dégrader significativement.

## 🧪 7) Évaluation optimisée — solver & variantes (lbfgs, saga, chi², SVD)
> Objectif : comparer l’effet **du solver** et de la **réduction de dimension** sur AP, F1, seuil optimal *(t\*)* et temps d’entraînement.

In [8]:
# === Évaluation "optimisée" sans toucher à la baseline existante ===
import time, numpy as np, pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, f1_score, precision_recall_curve
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.exceptions import ConvergenceWarning
import warnings

# (option) éviter d'inonder la sortie quand on compare plusieurs runs
warnings.filterwarnings("ignore", category=ConvergenceWarning)

def eval_pipeline_opt(Xtr, Xte, ytr, yte, desc: str,
                      solver="saga", C=0.5, max_iter=5000, tol=1e-3, penalty="l2"):
    t0 = time.time()
    clf = LogisticRegression(
        solver=solver, penalty=penalty, C=C,
        max_iter=max_iter, tol=tol,
        class_weight="balanced"
    )
    clf.fit(Xtr, ytr)
    fit_s = time.time() - t0

    proba = clf.predict_proba(Xte)[:, 1]
    ap = average_precision_score(yte, proba)
    prec, rec, thr = precision_recall_curve(yte, proba)
    f1 = (2*prec*rec)/(prec+rec+1e-9)
    best_idx = int(np.nanargmax(f1))
    thr_star = float(thr[best_idx]) if best_idx < len(thr) else 0.5
    f1_star = float(f1[best_idx])
    f1_05 = f1_score(yte, proba >= 0.5)
    print(f"[{desc}] AP={ap:.3f} | F1@0.5={f1_05:.3f} | F1@t*={f1_star:.3f} (t*={thr_star:.2f}) | fit={fit_s:.2f}s")
    return {"desc": desc, "AP": ap, "F1@0.5": f1_05, "F1@t*": f1_star, "t*": thr_star, "fit_s": fit_s}

# On part de Xtr, Xte, ytr, yte déjà construits par tes cellules précédentes
compare = []

# 1) Ta baseline existante (lbfgs) — on relance la même fonction que tu as déjà (si dispo)
try:
    compare.append(eval_pipeline(Xtr, Xte, ytr, yte, desc=f"Baseline lbfgs — p={Xtr.shape[1]:,}"))
except NameError:
    pass  # si ta fonction baseline s'appelle autrement, ignore

# 2) Même features mais solver 'saga' mieux adapté aux très grandes dimensions
compare.append(eval_pipeline_opt(Xtr, Xte, ytr, yte,
                                desc=f"saga C=0.5 tol=1e-3 — p={Xtr.shape[1]:,}",
                                solver="saga", C=0.5, max_iter=5000, tol=1e-3, penalty="l2"))

# 3) Variante rapide : pré-filtre chi² puis 'saga'
k = min(50_000, Xtr.shape[1])
sel = SelectKBest(chi2, k=k).fit(Xtr, ytr)
compare.append(eval_pipeline_opt(sel.transform(Xtr), sel.transform(Xte), ytr, yte,
                                desc=f"chi2(k={k:,}) + saga C=0.5", solver="saga", C=0.5))

# 4) (option) régularisation plus forte
compare.append(eval_pipeline_opt(Xtr, Xte, ytr, yte,
                                desc="saga C=0.25 tol=1e-3", solver="saga", C=0.25, max_iter=5000, tol=1e-3))

pd.DataFrame(compare).sort_values("AP", ascending=False)

[Baseline lbfgs — p=180,007] AP=0.983 | F1@0.5=0.927 | F1@t*=0.938 (t*=0.31) | fit=354.25s
[saga C=0.5 tol=1e-3 — p=180,007] AP=0.848 | F1@0.5=0.462 | F1@t*=0.865 (t*=0.04) | fit=453.03s
[chi2(k=50,000) + saga C=0.5] AP=0.848 | F1@0.5=0.462 | F1@t*=0.865 (t*=0.04) | fit=236.76s
[saga C=0.25 tol=1e-3] AP=0.848 | F1@0.5=0.462 | F1@t*=0.865 (t*=0.04) | fit=523.04s


,desc,AP,F1@0.5,F1@t*,t*,fit_s
0,"Baseline lbfgs — p=180,007",0.982659,0.927284,0.937503,0.309827,354.248536
3,saga C=0.25 tol=1e-3,0.847895,0.462044,0.864598,0.035994,523.043513
1,"saga C=0.5 tol=1e-3 — p=180,007",0.847894,0.462044,0.864598,0.035993,453.028244
2,"chi2(k=50,000) + saga C=0.5",0.847830,0.461813,0.864598,0.036054,236.760895


📏 Comprendre les métriques et lire nos résultats
1) Rappels de base

Précision (Precision) = TP / (TP + FP) : parmi ce que le modèle dit positif, quelle part est vraiment positive ?

Rappel (Recall) = TP / (TP + FN) : parmi tous les vrais positifs, quelle part a été retrouvée ?

F1-score = 2 × (Precision × Recall) / (Precision + Recall)
→ moyenne harmonique de P & R (punition si l’un des deux est bas).

Ces trois métriques dépendent d’un seuil sur la probabilité p̂(y=1).
Changer le seuil ⇢ changer la matrice de confusion ⇢ changer Precision/Recall/F1.

2) AP = Average Precision (aire sous la courbe PR)

L’AP est l’aire sous la courbe Precision–Recall quand on fait varier le seuil de 1 → 0.
Elle mesure la qualité de classement (le modèle met-il “les vrais positifs” en haut de la pile ?).

Seuil-libre : pas besoin de choisir un seuil pour la comparer entre modèles.

Référence : l’AP d’un classifieur aléatoire vaut la prévalence de la classe positive.
Ici, prévalence ≈ 76 % ⇒ un modèle nul aurait AP ≈ 0.76.

Lecture de nos chiffres :

lbfgs : AP = 0.983 👉 très au-dessus de 0.76 → excellente capacité à trier les avis.

saga (C=0.5) : AP = 0.848 👉 au-dessus de 0.76 mais nettement inférieur à lbfgs → le tri est moins bon.

Pourquoi AP est clé ici ? Avec un dataset déséquilibré (76 % positifs), la courbe PR et l’AP sont plus informatives que l’AUC-ROC.

3) F1@0.5 vs F1@t* (choix du seuil)

F1@0.5 : F1 au seuil 0.5 (par défaut).
Bon si les probabilités sont calibrées (≈ vrais risques) et si 0.5 a du sens métier.

F1@t* : F1 au meilleur seuil t* (celui qui maximise F1 sur l’échantillon évalué).
Utile pour voir le potentiel max du modèle, mais le choix final du seuil doit se faire sur un jeu de validation, pas sur le test.

Lecture de nos chiffres :

lbfgs : F1@0.5 = 0.927, F1@t* = 0.938 avec t* ≈ 0.31
→ Très bon F1 même à 0.5. Le meilleur seuil est un peu plus bas (0.31), ce qui est cohérent avec un dataset à 76 % positifs et l’effet de class_weight='balanced' qui déplace l’intercept.
→ Les probabilités sont bien étalées (bonne calibration relative).

saga (C=0.5) : F1@0.5 = 0.462, F1@t* = 0.865 avec t* ≈ 0.04
→ À 0.5, le modèle semble mauvais, mais si on baisse fortement le seuil, il remonte à F1@t* = 0.865.
→ t* très faible (0.04) = les probabilités sont compressées vers 0 → sous-apprentissage / sur-régularisation / calibration faible.
→ Même après optimisation du seuil, ça reste moins bon que lbfgs.

À retenir pour l’oral :

AP juge le classement global (seuil-free).

F1@0.5 montre la perf immédiate avec le seuil standard.

F1@t* montre le potentiel si on optimise le seuil (à faire sur validation, pas test).

4) Interprétation de t*

t* ≈ 0.31 (lbfgs) : pour maximiser F1, on classe positif dès ~31 % de proba.
→ logique avec une forte prévalence et des probas étalées.

t* ≈ 0.04 (saga) : il faut descendre très bas pour trouver les positifs → probas sous-estimées, indicateur de sous-apprentissage (régularisation trop forte C=0.5, tolérance large 1e-3, etc.).

5) Temps d’entraînement (fit)

lbfgs : ~354 s

saga C=0.5 : ~453–523 s (selon C)

chi²(k=50k) + saga : ~237 s
→ Le pré-filtre chi² divise le temps d’entraînement avec la même qualité que la variante saga C=0.5 (car même solveur/régularisation), mais reste loin derrière lbfgs en qualité.

🧭 Décision pour la suite

Objectif qualité → lbfgs + L2 (C=1.0) gagne nettement (AP/F1).
Si warning de convergence : augmenter max_iter (ex. 5000) et/ou relâcher tol (ex. 1e-3).

Objectif vitesse → appliquer chi²(k) ou SVD(k) avant la LogReg (idéalement avec lbfgs) pour réduire p et le temps, avec une faible perte attendue.

Seuil de décision : on choisit t sur un set de validation selon le besoin métier (par ex. privilégier la précision pour éviter les faux positifs, ou le rappel pour capter un max d’avis négatifs).

Calibration (option qualité produit) : mesurer le Brier score et appliquer une calibration (Platt / isotonic) si les proba sont mal étalées.

TL;DR :
lbfgs est meilleur pour ce corpus TF-IDF haute dimension (AP=0.983 ; F1@0.5=0.927).
Les variantes saga testées sous-apprennent (probas écrasées, t* minuscule) ; le chi² accélère mais n’égale pas lbfgs en qualité.

## 💾 8) Sérialisation des sélecteurs

In [22]:

# 8) Sérialisation des sélecteurs / réducteurs (safe)
from joblib import dump
from pathlib import Path
import time, json

stamp = time.strftime("%Y%m%d_%H%M%S")
out_dir = Path("models"); out_dir.mkdir(parents=True, exist_ok=True)

written = []

# Chi² (filtre)
if 'sel' in globals():
    p = out_dir / f"selector_chi2_k{int(sel.k)}_{stamp}.joblib"
    dump({"chi2": sel, "k": int(sel.k)}, p, compress=3)
    written.append(p.name)

# L1 (méthode incorporée après chi²)
if 'sfm' in globals():
    kept = int(Xtr_l1.shape[1]) if 'Xtr_l1' in globals() else None
    chi2_k = int(sel.k) if 'sel' in globals() else None
    p = out_dir / f"selector_l1_after_chi2_k{chi2_k}_kept{kept}_{stamp}.joblib"
    dump({"l1_sfm": sfm, "kept": kept, "chi2_k": chi2_k}, p, compress=3)
    written.append(p.name)

# SVD (réduction) — seulement si tu comptes le réutiliser
if 'svd_pipe' in globals():
    k = int(getattr(svd_pipe.named_steps['truncatedsvd'], 'n_components', 0))
    p = out_dir / f"reducer_svd_k{k}_{stamp}.joblib"
    dump({"svd_pipe": svd_pipe, "k": k}, p, compress=3)
    written.append(p.name)

# Trace lisible de ce qui a été écrit
(meta := {
    "created": stamp,
    "artifacts": written,
}).update({})
(out_dir / f"artifacts_{stamp}.json").write_text(json.dumps(meta, indent=2), encoding="utf-8")

print("Écrit →", written if written else "rien (exécute d'abord les cellules chi²/L1/SVD)")


Écrit → ['selector_chi2_k50000_20250912_215349.joblib', 'selector_l1_after_chi2_k50000_kept25000_20250912_215349.joblib', 'reducer_svd_k300_20250912_215349.joblib']


## 📊 9) Tableau comparatif synthèse — AP / F1 / t\* / temps
> On rassemble baseline et variantes dans un tableau unique avec **Δ vs baseline** pour justifier le choix final.

In [9]:
# === Tableau comparatif propre (baseline + variantes) ===
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

def run_case(Xtr, Xte, ytr, yte, desc, solver="lbfgs", C=1.0, max_iter=5000, tol=1e-3, penalty="l2"):
    out = eval_pipeline_opt(Xtr, Xte, ytr, yte, desc=desc,
                            solver=solver, C=C, max_iter=max_iter, tol=tol, penalty=penalty)
    out["p"] = Xtr.shape[1]
    return out

compare2 = []

# 0) Baseline explicite (lbfgs, sans sélection)
compare2.append(run_case(Xtr, Xte, ytr, yte,
                         desc=f"Baseline lbfgs — p={Xtr.shape[1]:,}",
                         solver="lbfgs", C=1.0, max_iter=5000, tol=1e-3, penalty="l2"))

# 1) Même features, solver 'saga' (pour montrer l'effet solver)
compare2.append(run_case(Xtr, Xte, ytr, yte,
                         desc="saga C=0.5 tol=1e-3",
                         solver="saga", C=0.5, max_iter=5000, tol=1e-3, penalty="l2"))

# 2) chi² + lbfgs (réduction p et comparaison à armes égales)
k = min(50_000, Xtr.shape[1])
sel = SelectKBest(chi2, k=k).fit(Xtr, ytr)
Xtr_chi, Xte_chi = sel.transform(Xtr), sel.transform(Xte)
compare2.append(run_case(Xtr_chi, Xte_chi, ytr, yte,
                         desc=f"chi²(k={k:,}) + lbfgs",
                         solver="lbfgs", C=1.0, max_iter=5000, tol=1e-3, penalty="l2"))

# 3) (option) SVD + lbfgs (autre façon de réduire p)
svd_pipe = make_pipeline(TruncatedSVD(n_components=300, random_state=42), Normalizer(copy=False))
Xtr_svd = svd_pipe.fit_transform(Xtr, ytr); Xte_svd = svd_pipe.transform(Xte)
compare2.append(run_case(Xtr_svd, Xte_svd, ytr, yte,
                         desc="SVD(300) + lbfgs",
                         solver="lbfgs", C=1.0, max_iter=5000, tol=1e-3, penalty="l2"))

# → Tableau + deltas vs baseline
dfc = pd.DataFrame(compare2)
base = dfc.iloc[0].copy()
for col in ["AP", "F1@0.5", "F1@t*", "fit_s"]:
    dfc[f"Δ{col}"] = dfc[col] - base[col]
dfc.rename(columns={"Δfit_s": "Δfit_s"}, inplace=True)  # alias cohérent

dfc[["desc","p","AP","ΔAP","F1@0.5","ΔF1@0.5","F1@t*","ΔF1@t*","fit_s","Δfit_s"]].sort_values("AP", ascending=False)

[Baseline lbfgs — p=180,007] AP=0.985 | F1@0.5=0.932 | F1@t*=0.942 (t*=0.31) | fit=914.02s
[saga C=0.5 tol=1e-3] AP=0.848 | F1@0.5=0.462 | F1@t*=0.865 (t*=0.04) | fit=464.46s
[chi²(k=50,000) + lbfgs] AP=0.984 | F1@0.5=0.928 | F1@t*=0.941 (t*=0.31) | fit=371.32s
[SVD(300) + lbfgs] AP=0.849 | F1@0.5=0.504 | F1@t*=0.865 (t*=0.26) | fit=0.38s


,desc,p,AP,ΔAP,F1@0.5,ΔF1@0.5,F1@t*,ΔF1@t*,fit_s,Δfit_s
0,"Baseline lbfgs — p=180,007",180007,0.984606,0.000000,0.932396,0.000000,0.941805,0.000000,914.018394,0.000000
2,"chi²(k=50,000) + lbfgs",50000,0.983798,-0.000808,0.927978,-0.004417,0.941151,-0.000654,371.321419,-542.696974
3,SVD(300) + lbfgs,300,0.849346,-0.135260,0.504495,-0.427900,0.864577,-0.077228,0.383034,-913.635359
1,saga C=0.5 tol=1e-3,180007,0.847895,-0.136711,0.462044,-0.470351,0.864598,-0.077207,464.457866,-449.560527


📌 Lecture des 4 variantes

Baseline (TF-IDF complet → LogReg lbfgs)
AP=0.985 | F1@0.5=0.932 | F1@t*=0.942 (t*=0.31) | fit≈914s
→ Meilleure qualité globale. t*≈0.31 = proba bien étalées (calibration relative OK).

SAGA (C=0.5, tol=1e-3)
AP=0.848 | F1@0.5=0.462 | F1@t*=0.865 (t*=0.04) | fit≈464s
→ Forte baisse de qualité (AP, F1). t* minuscule ⇒ proba compressées vers 0 ⇒ sous-apprentissage / régularisation trop forte.

chi²(k=50k) + lbfgs
AP=0.984 | F1@0.5=0.928 | F1@t*=0.941 (t*=0.31) | fit≈371s
→ Quasi identique à la baseline (ΔAP ≈ -0.001 ; ΔF1@0.5 ≈ -0.004), mais 2.5× plus rapide et beaucoup moins de variables (p=50k vs ~180k). t* inchangé ⇒ calibration préservée.
=> Excellent compromis qualité/vitesse.

SVD(300) + lbfgs
AP=0.849 | F1@0.5=0.504 | F1@t*=0.865 (t*=0.26) | fit≈0.38s
→ Ultra-rapide (≈ 2400× plus vite) mais perte de qualité marquée (ΔAP ≈ -0.136). À réserver si la contrainte temps est extrême (ex : prototypage, pré-tri).

🎯 Décision pour la suite (C5.2.3)

Pipeline retenu (MVP) : TF-IDF (word+char) → chi²(k=50k) → LogReg (L2, lbfgs, C=1.0)
Motifs : qualité quasi identique à la baseline, fit 2.5× plus rapide, mémoire réduite, calibration conservée (t* ≈ 0.31).

Seuil : on choisira t sur un set de validation selon le besoin métier (précision vs rappel).

Pistes optionnelles : tester k=80k/100k pour voir si ΔAP devient nul ; ou SVD(k=500–1000) si on veut explorer un compromis vitesse/qualité entre 300 et TF-IDF complet.

In [10]:
dfc.to_csv("tableau_comparatif_baseline_vs_variantes.csv")

« Nous avons évalué trois familles : filtre (chi²), réduction (SVD) et incorporée (L1).
La méthode incorporée sélectionne les variables durant l’apprentissage (L1 met des poids à zéro) → elle modifie l’espace de features et produit un masque réutilisable.
L’évaluation optimisée, elle, n’est pas une méthode de sélection : elle compare des configurations d’entraînement (solver, régularisation) sur un même espace de features (ou après un filtre).
Sur nos données, TF-IDF → chi²(k=50k) → LogReg lbfgs offre la meilleure qualité/vitesse. Nous conservons L1 en démonstration méthodologique pour valider la compétence “méthode incorporée”. »

En une phrase chacune

- chi² : “Je trie vite, indépendamment du modèle.”

- SVD : “Je compresse en thèmes.”

- L1 incorporée : “J’apprends et j’élimine des mots pendant l’entraînement.”

- Évaluation optimisée : “Je règle le modèle sur un même jeu de mots.”

Pourquoi L1 est plus lente ?

- Parce qu’elle entraîne un modèle entier pour décider quoi couper (et en TF-IDF avec des dizaines de milliers de mots, c’est un gros jardin 😅).
Chi², lui, regarde chaque mot séparément : c’est un check binaire ultra rapide.

“On garde TF-IDF → chi²(50k) → LogReg lbfgs : même qualité que la baseline, beaucoup plus rapide.
On montre L1 en méthode incorporée pour prouver qu’on sait sélectionner en apprenant, mais on ne la met pas en prod car elle est plus lente pour peu de gain ici.”

« On utilise le SVD quand on veut compresser le TF-IDF en quelques centaines de dimensions, pour aller très vite ou travailler sur des thèmes latents (LSA).
C’est parfait pour explorer/visualiser ou déployer léger, mais on accepte une petite perte de précision.
Dans notre cas, le meilleur compromis production est TF-IDF → chi²(50k) → LogReg. On garde SVD pour l’exploration et les cas contraints (vitesse/mémoire) ou pour des modules annexes (clustering, recherche sémantique). »

“La LogReg-L1 peut remplacer le duo sélection (chi²) + classifieur, car elle sélectionne et classifie en même temps. En revanche, elle ne remplace pas SVD, qui réalise une compression en nouvelles composantes. Pour nos données, nous retenons TF-IDF → chi²(k) → LogReg (L2, lbfgs) pour le meilleur compromis qualité/temps, et nous gardons L1 comme démonstration de méthode incorporée.”


---

## ✅ Rappel à la compétence & pourquoi c’est validé
**Compétence :** *C5.2.2 — Sélection de variables*  
- On a implémenté **plusieurs méthodes** (test **chi²**, **L1** embarqué, **SVD**, *(option RFE)*).  
- On a **comparé** sur un split stratifié avec **AP/F1** et **coût** (fit), et rapporté le **nombre de variables** retenues.  
- On a **sérialisé** les sélecteurs pour réutilisation dans l'entraînement (C5.2.3) et l’inférence.  
- On a **interprété** les features top (chi²/L1) pour documenter la pertinence de la liste.

À quoi servent les « sélecteurs » (de variables) en général ?

But principal : choisir un sous-ensemble de features utiles et jeter le reste.
Ça sert à :

Mieux généraliser

Moins de bruit ⇒ moins de surapprentissage ⇒ scores plus stables.

Accélérer et réduire la mémoire

Moins de colonnes ⇒ fit/prédiction plus rapides, modèles plus légers.

Améliorer l’interprétabilité

On garde les signaux clés ⇒ plus facile d’expliquer pourquoi le modèle décide.

Gérer la colinéarité

Features redondantes ou très corrélées ⇒ on en garde une partie.

Respecter des contraintes

Coûts d’acquisition, RGPD/PII, limites de calcul en prod, etc.

Trois familles (et quand les utiliser)

Filtres (indépendants du modèle) – ex. chi², mutual information

Rapides, simples, scalables.

Idéal en pré-filtre quand p est énorme (texte TF-IDF).

Ne voient pas les interactions.

Méthodes incorporées (embedded) – ex. LogReg L1, arbres (importances)

La sélection se fait pendant l’apprentissage.

Plus pertinentes pour le modèle ciblé, mais plus lentes.

Wrappers – ex. RFE

Testent des sous-ensembles avec un modèle “dans la boucle”.

Précis mais coûteux → plutôt sur sous-espaces.

(Rappel : SVD/LSA n’est pas un sélecteur, c’est une réduction qui crée de nouvelles composantes.)

Bonnes pratiques (anti-boulettes)

Toujours fitter le sélecteur sur le train uniquement (puis transformer le test) → éviter la fuite de données.

Choisir k (nombre de features) par validation croisée ou sur un set de validation.

Comparer à une baseline sans sélection (montrer ΔAP/ΔF1/Δtemps).

Sur texte TF-IDF : commencer par chi²(k), puis éventuellement affiner (L1) si besoin.

Petite image pour retenir

chi² = videur : regarde chaque feature à l’entrée et dit “tu rentres / tu rentres pas”.

L1 = jardinier qui taille pendant la course : le modèle apprend et coupe les branches inutiles.

SVD = blender : compresse tous les ingrédients en quelques “saveurs” (composantes) — ce n’est pas une sélection.

Dans TON projet

Tu utilises les sélecteurs pour :
(a) garder le signal utile (meilleure généralisation),
(b) réduire p ⇒ fit 2–3× plus rapide,
(c) présenter des termes clés (interprétation).

Le compromis gagnant que tu as montré : TF-IDF → chi²(k=50k) → LogReg (lbfgs) : même qualité que la baseline, beaucoup plus rapide.

L1 est gardée comme démonstration de méthode incorporée (sélection pendant l’apprentissage).